In [9]:
import mysql.connector

In [10]:
config = {
    'user':'root',
    'password':'password',
    'host':'localhost',
    'database':'',
    'port':'3306',
}

In [11]:
def connect(config):
    return mysql.connector.connect(**config)

In [12]:
cnx = connect(config)
cursor = cnx.cursor()

In [15]:
q = "USE Chinook"
cursor.execute(q)

In [19]:
q = "SHOW tables"
cursor.execute(q)
cursor.fetchall()

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [24]:
q = "DESCRIBE InvoiceLine"
cursor.execute(q)
cursor.fetchall()

[('InvoiceLineId', 'int', 'NO', 'PRI', None, ''),
 ('InvoiceId', 'int', 'NO', 'MUL', None, ''),
 ('TrackId', 'int', 'NO', 'MUL', None, ''),
 ('UnitPrice', 'decimal(10,2)', 'NO', '', None, ''),
 ('Quantity', 'int', 'NO', '', None, '')]

#### Write a query that gathers data about the invoice with an invoice_id of 4. Include the following columns in order:
* The id of the track, track_id.
* The name of the track, track_name.
* The name of media type of the track, track_type.
* The price that the customer paid for the track, unit_price.
* The quantity of the track that was purchased, quantity.

In [25]:
q = """
SELECT * FROM InvoiceLine il
INNER JOIN Track t ON t.TrackId = il.TrackId
INNER JOIN MediaType mt ON mt.MediaTypeId = t.MediaTypeId
WHERE il.InvoiceLineId = 3;
"""
cursor.execute(q)
r = cursor.fetchall()

for i in r:
    print(i)

(3, 2, 6, Decimal('0.99'), 1, 6, 'Put The Finger On You', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 205662, 6713451, Decimal('0.99'), 1, 'MPEG audio file')
